🧩 Step 0: Install Required Libraries
Install all the dependencies needed to:

Load PDFs

Chunk and embed documents

Store and retrieve vectors in Pinecone

Generate answers using Gemini LLM

In [1]:
# 📦 STEP 0: Install Required Libraries
# These commands install all needed packages in the Colab environment.
# - langchain-community: for loading and chunking PDF documents
# - google-generativeai: for embedding and generation using Gemini
# - pinecone: for storing and retrieving semantic vectors
# - pypdf: backend library used by LangChain to parse PDFs
!pip install -U langchain-community
!pip install -U google-generativeai
!pip install -U pinecone
!pip install -U pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.9/517.9 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 21.4 MB/s eta 0:00:00


📁 Step 1: Upload and Load PDF
This allows the user to upload a .pdf file and load it using LangChain's PyPDFLoader.
Each page is turned into a document chunk for easier processing later.

In [2]:
# 📄 STEP 1: Upload and Load PDF using LangChain
# This cell allows the user to upload a PDF document manually.
# PyPDFLoader reads the uploaded PDF and loads it into LangChain document format—
# where each page becomes a document. These will be split into chunks in the next step.
from langchain_community.document_loaders import PyPDFLoader
from google.colab import files

uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]

# Load the PDF into document objects
loader = PyPDFLoader(pdf_path)
documents = loader.load()

print(f"✅ Loaded {len(documents)} pages from the PDF")

Saving Weekly Schedule - AI_ML with Python.pdf to Weekly Schedule - AI_ML with Python (1).pdf
✅ Loaded 1 pages from the PDF


✂️ Step 2: Split Text into Chunks
Use LangChain’s RecursiveCharacterTextSplitter to break each page into overlapping text blocks.
Chunking improves retrieval accuracy and preserves context.

In [3]:
# ✂️ STEP 2: Split PDF Text into Smaller Chunks
# Splitting long documents into manageable chunks ensures better performance in retrieval and embedding.
# chunk_size=500 ensures each chunk is 500 characters long,
# chunk_overlap=100 means 100 characters are shared between consecutive chunks for context retention.
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)
chunks = splitter.split_documents(documents)

print(f"✅ Split the document into {len(chunks)} chunks")

✅ Split the document into 4 chunks


🔐 Step 3A: Set Your Gemini API Key
This key allows access to Google’s Gemini embedding and generation services.


In [4]:
# 🔐 STEP 3A: Set Gemini API Key (Google Generative AI)
# The Gemini API key is required to use both the embedding and language model endpoints.
import os
os.environ["GOOGLE_API_KEY"] = "YOUR_GEMINI_API_KEY"

🔡 Step 3B: Embed Chunks Using Gemini (text-embedding-004)
Convert each text chunk into a semantic vector using Gemini’s latest embedding model.
These vectors will be stored in Pinecone for fast, relevant retrieval.

In [5]:
# 🧠 STEP 3B: Generate Embeddings for Each Chunk using Gemini Embedding API
# This step converts every document chunk into a dense vector using Gemini's text embedding model.
# These vectors will later be stored in Pinecone and used for semantic similarity search.
import google.generativeai as genai

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

embedding_model = genai.embed_content
texts = [chunk.page_content for chunk in chunks]

embedded_texts = [
    embedding_model(
        model="models/text-embedding-004",
        content=text,
        task_type="retrieval_document"
    )["embedding"] for text in texts
]

print(f"✅ Generated embeddings for {len(embedded_texts)} chunks")

✅ Generated embeddings for 4 chunks


🔐 Step 4A: Set Pinecone API Key and Region
Set your Pinecone API key and select a region.


In [6]:
# 🔐 STEP 4A: Set Pinecone API Key and Region
# Pinecone is used to store and query the embeddings.
os.environ["PINECONE_API_KEY"] = "YOUR_PINECONE_API_KEY"

📦 Step 4B: Create Pinecone Index & Upload Vectors
Connect to Pinecone, create an index (if not already created), and upload all embeddings with their text chunks.

In [7]:
# 📦 STEP 4B: Create Pinecone Index and Upload Vectors
# Here we initialize Pinecone, create an index (if not already created), and upload our vectors.
# Index stores each chunk vector along with the original text in metadata for retrieval.
from pinecone import Pinecone, ServerlessSpec
from uuid import uuid4

# Initialize Pinecone client
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])

index_name = "rag-gemini-index"
dimension = len(embedded_texts[0])

# Create index only if it doesn't already exist
if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=dimension,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
    print(f"✅ Created new index: {index_name}")

# Connect to the index and upload vectors
index = pc.Index(index_name)

vectors = [
    {
        "id": str(uuid4()),
        "values": vector,
        "metadata": {"text": text}
    }
    for text, vector in zip(texts, embedded_texts)
]

index.upsert(vectors=vectors)
print(f"✅ Uploaded {len(vectors)} vectors to Pinecone")

✅ Uploaded 4 vectors to Pinecone


❓ Step 5: Embed a User Query and Retrieve Matching Chunks
Ask a natural language question. It will be embedded and used to find similar chunks from the Pinecone vector index.

In [8]:
# 🧠 STEP 5: Embed a User Question and Search Pinecone for Similar Chunks
# This is the 'Retrieval' part of RAG: embed a user query, search the vector DB, and retrieve top matching chunks.
question = "Summarize the document in 3 lines."

query_embedding = embedding_model(
    model="models/text-embedding-004",
    content=question,
    task_type="retrieval_query"
)["embedding"]

search_results = index.query(
    vector=query_embedding,
    top_k=5,
    include_metadata=True
)

matched_chunks = [match["metadata"]["text"] for match in search_results["matches"]]
print("🔍 Retrieved context chunks:")
for chunk in matched_chunks:
    print("•", chunk[:150], "...\n")

🔍 Retrieved context chunks:
• (26th June, 2025)
Week 10
(17th July, 2025)
AI/ML Using Python - Course Schedule
Assignment - 3
Assignment-4 -Practice Case Study
 (Stock file)
Assign ...

• Week Topic Assignment 
Introduction to DS
Introduction to Python
Introduction to Spyder I & II
Variables & DataTypes
Operators
Sequence Data Types [1- ...

• Week 11
(24th July, 2025)
Week 12
(31st July, 2025)
Week 1 
(15th May, 2025)
Week 2 
(22nd May, 2025)
Week 3 
(29th May, 2025)
Week 4
(5th June, 2025) ...

• Hands-on Optimization
Webinar - Debugging
Week 8
(03rd July, 2025) Predictive Modeling -1
Week 9
(10th July, 2025) Predictive Modeling - 2 Assignment  ...

• - Precise type annotations
- New performance improvements and error messages
Visit: https://docs.python.org/3/library/functions.html for the full list ...



🧠 Step 6: Answer the Question Using Gemini LLM (gemini-1.5-flash)
Generate a natural language answer based on retrieved context using Gemini’s text model.

In [9]:
# 🧠 STEP 6: Use Gemini to Answer the Question Based on Retrieved Chunks
# This is the 'Generation' part of RAG: generate a final answer using the retrieved text + original question.
llm_model = genai.GenerativeModel("models/gemini-1.5-flash")

context = "\n\n".join(matched_chunks)

prompt = f"""You are a helpful assistant. Based on the following context, answer the user's question.

Context:
{context}

Question: {question}

Answer:"""

response = llm_model.generate_content(prompt)

print("🧠 Gemini's Answer:\n")
print(response.text)


🧠 Gemini's Answer:

This document is a course schedule for an AI/ML using Python course running from May 15th to July 31st, 2025.  The course covers topics from introductory data science and Python to advanced machine learning techniques.  Assignments include quizzes, case studies, and hands-on exercises.

